<a href="https://colab.research.google.com/github/aleks-haksly/VIZRO/blob/main/demos/bugfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
import pandas as pd
from google.colab import userdata
import ipywidgets as widgets
from IPython.display import display
import datetime
engine = create_engine(userdata.get('supabase'), client_encoding='utf8', )

def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

In [2]:
start_date = widgets.DatePicker(description='Start Date', disabled=False, value = datetime.date(2021,9,1))
end_date = widgets.DatePicker(description='End Date', disabled=False, value = datetime.date(2021,9,21))
limit = widgets.IntText(description='Limit', value=100, )
button = widgets.Button(description="Apply")
output = widgets.Output()

In [3]:
SQL_TEMPLATE_QUERY_LINECHART = f"""
WITH t AS (
    SELECT
        query,
        ROW_NUMBER() OVER (PARTITION BY date_part('hour', ts) ORDER BY COUNT(*) DESC) AS rnb
    FROM
        vizro.yandex_data yd
    WHERE
        ts BETWEEN '{start_date.value.strftime('%Y-%m-%d')}' AND '{end_date.value.strftime('%Y-%m-%d')}'
    GROUP BY query, date_part('hour', ts)
)
SELECT
    platform,
    date_part('hour', ts)::INT AS hour,
    query,
    COUNT(*)
FROM
    vizro.yandex_data yd
WHERE
    query IN (SELECT DISTINCT query FROM t WHERE rnb < 6)
GROUP BY
    platform, date_part('hour', ts), query
    {f'LIMIT {limit.value}' if limit.value else ''}
"""

In [9]:
def on_button_clicked(b):
    df = select(SQL_TEMPLATE_QUERY_LINECHART)
    with output:
        df = select(SQL_TEMPLATE_QUERY_LINECHART)
        output.clear_output()
        display(df)
    #return df
button.on_click(on_button_clicked)

In [10]:
display(start_date, end_date, limit, button, output)

DatePicker(value=datetime.date(2021, 9, 1), description='Start Date')

DatePicker(value=datetime.date(2021, 9, 21), description='End Date')

IntText(value=100, description='Limit')

Button(description='Apply', style=ButtonStyle())

Output()

In [12]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [15]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
function_with_captured_output()

In [14]:
function_with_captured_output()